## General methods

In [ ]:
import numpy as np
import pandas as pd
import gdown
import datetime as dt
import sqlite3
import kaggle
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

### Download Kaggle dataset

In [ ]:
def kaggle_dataset_download(kaggle_path, kaggle_path_Name, kaggle_zip_file):
    kaggle.api.authenticate()
    if kaggle_zip_file:
        kaggle.api.dataset_download_files(kaggle_path, kaggle_path_Name, unzip=True)
    else:
        kaggle.api.dataset_download_files(kaggle_path, kaggle_path_Name)